In [ ]:
from selenium import webdriver
import time
import pandas as pd
import os
from bs4 import BeautifulSoup as bs

# Link to the page
link = "https://www.cars24.com/buy-used-car?sort=bestmatch&serveWarrantyCount=true&storeCityId=4709"

# Initialize the Chrome WebDriver
browser = webdriver.Chrome()

try:
    browser.get(link)
    time.sleep(3)  # Let the page load

    # Scrolling down to the bottom of the page to load all data
    ini_height = browser.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll to the bottom
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Pause to allow content to load

        # Calculate new scroll height and compare with the previous height
        new_height = browser.execute_script("return document.body.scrollHeight")

        if new_height == ini_height:
            break
        ini_height = new_height
        time.sleep(3)

    # After scrolling, get the final page source
    html = browser.page_source

    # Use BeautifulSoup to parse the page source
    soup = bs(html, 'html.parser')

    # Find all car listings
    cars = soup.find_all('div', class_="_7jb8Q _1Ey60")

    # Output the number of cars found
    print(f"Number of cars found: {len(cars)}")

    # Lists to store data
    year_manu_list = []
    brand_list = []
    name_list = []
    km_tra_list = []
    fuel_list = []
    price_list = []

    # Extract car details
    for car in cars:
        name = car.find('h3').get_text()
        year_manu = name.split()[0]
        brand = name.split()[1]
        name = " ".join(name.split()[1:])
        ul = car.find("ul")
        lists = ul.find_all('li')
        km_tra = lists[1].get_text()
        fuel = lists[2].get_text()
        price = car.find('strong').get_text()

        # Append data to respective lists
        year_manu_list.append(year_manu)
        brand_list.append(brand)
        name_list.append(name)
        km_tra_list.append(km_tra)
        fuel_list.append(fuel)
        price_list.append(price)

    # Create a DataFrame
    dataset = pd.DataFrame({
        'Name': name_list,
        'Brand Name': brand_list,
        'Model Year': year_manu_list,
        'Distance Driven': km_tra_list,
        'Fuel Type': fuel_list,
        'Price': price_list
    })

    # Close the browser
    browser.quit()

    # Save the dataset to CSV
    output_path = os.path.join(os.getcwd(), 'cars24blr.csv')
    dataset.to_csv(output_path, index=False)

    # Confirmation message with file path
    print(f"CSV file saved successfully at: {output_path}")

except Exception as e:
    # Print any error that occurs during the scraping or saving process
    print(f"An error occurred: {e}")
    browser.quit()
